In [2]:
import requests

In [72]:
class Reg_API:
    def __init__(self, page_size=20):
        self.apibase = "https://api.regulations.gov/v4"
        self.apikey = "e7LVpmbLfa0f0dDAx6TPzg86cG5TASGTafkxQHWg"
        self.page_size = page_size
        self.reqstr = ""

    def _add_apikey(self):
        self.reqstr = self.reqstr + f"?api_key={self.apikey}"
        return self

    ### Main url constructors
    def endpoint(self, endpoint):
        self.reqstr = f"{self.apibase}{endpoint}"
        self._add_apikey()
        return self

    def url(self, url):
        self.reqstr = url
        self._add_apikey()
        return self

    ### Search modifiers
    def search(self, search_term):
        self.reqstr = self.reqstr + f"&filter[searchTerm]={search_term}"
        return self

    def page(self, page):
        self.reqstr = self.reqstr + f"&page[size]={self.page_size}&page[number]={page}"
        return self

    ### Get response(s)
    def get(self, get_json=True):
        response = requests.get(self.reqstr)
        if get_json:
            response = response.json()
        self.reqstr = ""
        return response

    def get_all(self):
        responses = []
        page = 1
        while True:
            response = requests.get(self.page(page).reqstr).json()
            responses.append(response)
            if not response["meta"]["hasNextPage"]:
                break
        self.reqstr = ""
        return responses

In [73]:
api = Reg_API()
comment_data = api.endpoint("/comments").search("climate").get()
comment_links = [ comment["links"]["self"] for comment in comment_data["data"]]
sample_comment = api.url(comment_links[0]).get()

In [87]:
comment_data['data'][0]

{'id': 'PHMSA-2019-0100-2389',
 'type': 'comments',
 'attributes': {'documentType': 'Public Submission',
  'lastModifiedDate': '2019-08-06T13:57:16Z',
  'highlightedContent': 'Comment from <mark><em>Climate</em></mark> Writers',
  'withdrawn': False,
  'agencyId': 'PHMSA',
  'title': 'Comment from Climate Writers',
  'objectId': '0900006483dd9a06',
  'postedDate': '2019-08-02T04:00:00Z'},
 'links': {'self': 'https://api.regulations.gov/v4/comments/PHMSA-2019-0100-2389'}}

Parse `["links"]["self"]` to get to comment data

In [77]:
sample_comment

{'data': {'id': 'PHMSA-2019-0100-2389',
  'type': 'comments',
  'links': {'self': 'https://api.regulations.gov/v4/comments/PHMSA-2019-0100-2389'},
  'attributes': {'commentOn': '0900006483cf03d6',
   'commentOnDocumentId': 'PHMSA-2019-0100-0002',
   'duplicateComments': 0,
   'address1': None,
   'address2': None,
   'agencyId': 'PHMSA',
   'city': None,
   'category': None,
   'comment': 'attach',
   'country': None,
   'displayProperties': None,
   'docAbstract': None,
   'docketId': 'PHMSA-2019-0100',
   'documentType': 'Public Submission',
   'email': None,
   'fax': None,
   'field1': None,
   'field2': None,
   'fileFormats': None,
   'firstName': 'Climate',
   'govAgency': None,
   'govAgencyType': None,
   'objectId': '0900006483dd9a06',
   'lastName': 'Writers',
   'legacyId': None,
   'modifyDate': '2019-08-06T13:57:16Z',
   'organization': None,
   'originalDocumentId': None,
   'pageCount': 0,
   'phone': None,
   'postedDate': '2019-08-02T04:00:00Z',
   'postmarkDate': Non

### Main Data

- comment: *comment text*

### Fields to consider

Categories

 - **address info**: address1, address2, city, stateProvinceRegion, country, zip
 - **date info**: modifyDate, postedDate, postmrkDate, receiveDate
 - **submitter info**: firstName, lastName, organization, govAgency, govAgencyType

Other

- title
- commentOnDocumentId
- duplicateComments
- category, documentType, subtype: *agency specific*
- docAbstract: *The detailed description of the document. This is an agency configurable field. An agency can configure this field to make it not publicly accessible*
- docketId

Parse `["relationships"]["attachments"]["links"]['related']` to get to attachment data if needed

In [80]:
sample_attachment = api.url(sample_comment["data"]["relationships"]["attachments"]["links"]['related']).get()
sample_attachment

{'data': [{'id': '0900006483dd9a07',
   'type': 'attachments',
   'links': {'self': 'https://api.regulations.gov/v4/attachments/0900006483dd9a07'},
   'attributes': {'agencyNote': None,
    'authors': None,
    'docAbstract': None,
    'docOrder': 1,
    'fileFormats': [{'fileUrl': 'https://downloads.regulations.gov/PHMSA-2019-0100-2389/attachment_1.pdf',
      'format': 'pdf',
      'size': 446371}],
    'modifyDate': '2019-07-31T15:55:21Z',
    'publication': None,
    'restrictReason': None,
    'restrictReasonType': None,
    'title': 'Comment from Climate Writers'}}]}

Download and parse `["fileFormats"][n]["fileUrl"]`
- Can include hand-written notes/pdfs.
- OCR tools like `pytesseract`?

### Fields to consider

Most fields here are duplicates of the parent "comment" data

- agencyNote


In [85]:
sample_attachment_file = api.url(sample_attachment['data'][0]['attributes']["fileFormats"][0]["fileUrl"]).get(get_json=False)
with open("test.pdf", "wb") as f:
    f.write(sample_attachment_file.content)